# Retrieving calibration data from Quantum Inspire

Pieter Eendebak <pieter.eendebak@tno.nl>

- Intro ...
- Compilation and transpilation options not available at this moment. See knowledge base for details
- Calibrations are coupled to experiments. Even though the calibrations are done on a periodic basis (e.g. once per day), every experiment gets mapped to such a calibration.
- Calibrations are backend specific

In [1]:
import numpy as np
from typing import Any

from qiskit import execute
from qiskit.circuit import QuantumCircuit

from quantuminspire.api import QuantumInspireAPI
from quantuminspire.credentials import \
    get_authentication, \
    get_token_authentication, \
    load_account, \
    save_account
from quantuminspire.qiskit import QI

import quantuminspire.calibration_viewer
from importlib import reload
reload(quantuminspire.calibration_viewer)
from quantuminspire.calibration_viewer import CalibrationViewer

try:
    from rich import print as rprint
except ImportError:
    rprint = print

In [2]:
def remove_ordered_dict(d: Any) -> Any:
    """ Replace OrderedDict with dict """
    if isinstance(d, dict):
        return {k: remove_ordered_dict(v) for k, v in d.items()}
    else:
        return d

## Set credentials

In [3]:
authentication = get_authentication()
QI.set_authentication(authentication)
qi_api = QI.get_api()
backend = QI.get_backend("Starmon-5")
# quantum_instance = QuantumInstance(backend=backend, shots=2**10)

In [4]:
QI_URL = r'https://api.quantum-inspire.com'
token = load_account()
authentication = get_token_authentication(token)
qi_api = QuantumInspireAPI(QI_URL, authentication)

QI.set_authentication(authentication, QI_URL)

## Run an experiment to generate data

In [5]:
backend_name = 'Spin-2'
if qi_api.get_backend_type_by_name(backend_name)["status"] == "OFFLINE":
        backend_name = 'Starmon-5'
qi_backend = QI.get_backend(backend_name)

print(f'Selected backend: {backend_name}')

circuit = QuantumCircuit(2)

circuit.h(1)
circuit.cx(1, 0)
circuit.measure_all()

qi_job = execute(circuit, backend=qi_backend, shots=256)
print(f'Execute job on {qi_backend}')
qi_result = qi_job.result()
histogram = qi_result.get_counts(circuit)
print(f'Measured result: {histogram}')

Selected backend: Starmon-5
Execute job on Starmon-5
Measured result: {'00': 133, '01': 13, '10': 7, '11': 103}


# Select job from backend to get calibration data from

In [6]:
qi_backend = QI.get_backend(backend_name)
all_jobs = qi_api.get_jobs()
backend_types = qi_api.get_backend_types()
backends = qi_api._get('https://api.quantum-inspire.com/backends')

backend_url = [b for b in backends if b['name']==backend_name][0]['url']
backend_jobs = [j for j in all_jobs if j['backend']==backend_url]
print(f"Found {len(backend_jobs)} jobs for backend {backend_name}.")

if len(backend_jobs)==0:
    raise Exception(f"You have not executed jobs in {backend_name}, no calibration data available.")

Found 314 jobs for backend Starmon-5.


In [7]:
qi_backend.__dict__

{'_configuration': <qiskit.providers.models.backendconfiguration.QasmBackendConfiguration at 0x7f74700dfdf0>,
 '_options': namespace(shots=1024, memory=True),
 '_provider': <quantuminspire.qiskit.quantum_inspire_provider.QuantumInspireProvider at 0x7f74201fc580>,
 '_QuantumInspireBackend__backend': {'url': 'https://api.quantum-inspire.com/backendtypes/11/',
  'name': 'Starmon-5',
  'is_hardware_backend': True,
  'required_permission': 'can_execute',
  'number_of_qubits': 5,
  'description': '',
  'topology': OrderedDict([('edges', [[2], [2], [0, 1, 3, 4], [2], [2]])]),
  'is_allowed': True,
  'status': 'EXECUTION',
  'status_message': '',
  'chip_image_id': '4386293',
  'calibration': 'https://api.quantum-inspire.com/backendtypes/11/calibration/',
  'allowed_operations': OrderedDict([('display', []),
               ('measure', ['measure', 'measure_x', 'measure_y', 'measure_z']),
               ('measure_all', ['measure_all']),
               ('parameterized_single_gates', ['rx', 'ry', 

In [8]:
# all_jobs
backends

[OrderedDict([('url', 'https://api.quantum-inspire.com/backends/1/'),
              ('name', 'qxsim'),
              ('location', 'Delft'),
              ('backend_type',
               'https://api.quantum-inspire.com/backendtypes/1/'),
              ('status', 'IDLE'),
              ('status_message', ''),
              ('lastHeartbeat', '2021-06-14T08:09:04.851276Z'),
              ('openqlPdf', ''),
              ('calibration',
               'https://api.quantum-inspire.com/backends/1/calibration/')]),
 OrderedDict([('url', 'https://api.quantum-inspire.com/backends/7/'),
              ('name', 'Spin-2'),
              ('location', 'Spitfire'),
              ('backend_type',
               'https://api.quantum-inspire.com/backendtypes/10/'),
              ('status', 'CALIBRATION'),
              ('status_message', ''),
              ('lastHeartbeat', '2021-06-25T12:29:03.153227Z'),
              ('openqlPdf', ''),
              ('calibration',
               'https://api.quantum-i

In [9]:
backend_types

[OrderedDict([('url', 'https://api.quantum-inspire.com/backendtypes/10/'),
              ('name', 'Spin-2'),
              ('is_hardware_backend', True),
              ('required_permission', 'can_execute'),
              ('number_of_qubits', 2),
              ('description', ''),
              ('topology', OrderedDict([('edges', [[1], [0]])])),
              ('is_allowed', True),
              ('status', 'OFFLINE'),
              ('status_message', 'This backend is offline.'),
              ('chip_image_id', '956681'),
              ('calibration',
               'https://api.quantum-inspire.com/backendtypes/10/calibration/'),
              ('allowed_operations',
               OrderedDict([('display', []),
                            ('measure',
                             ['measure',
                              'measure_z',
                              'measure_x',
                              'measure_y']),
                            ('measure_all', ['measure_all']),
        

In [11]:
# select latest job (appears first in list of jobs)
job = backend_jobs[0]
print(f"Found job with id {job['id']}")
result = qi_api.get_result_from_job(job['id'])
calibration = qi_api.get_calibration_from_result(result['id'])
print(calibration)

calibration = remove_ordered_dict(calibration)
print(calibration)
date = calibration['parameters']['system']['last_calibration_date']['value']
print(f'Last calibration data from {date}')

Found job with id 7076794
OrderedDict([('url', 'https://api.quantum-inspire.com/calibration/109006/'), ('backend', 'https://api.quantum-inspire.com/backends/8/'), ('parameters', OrderedDict([('system', OrderedDict([('fridge_temperature', OrderedDict([('value', '20.9'), ('unit', 'mK'), ('symbol', 'Fridge temperature')])), ('last_calibration_date', OrderedDict([('value', '04/11/2021 - 16:42 PM'), ('unit', 'iso-8601'), ('symbol', 'Calibration date')]))])), ('qubits', OrderedDict([('q0', OrderedDict([('t1', OrderedDict([('value', 1.7341880614618646e-05), ('unit', 's'), ('symbol', 'T1'), ('decimals', 1)])), ('t2star', OrderedDict([('value', 2.6163761239800992e-05), ('unit', 's'), ('symbol', 'T2e'), ('decimals', 1)])), ('single_qubit_gate_fidelity', OrderedDict([('value', 99.9), ('unit', '%'), ('symbol', 'F1q')])), ('two_qubit_gate_fidelity', OrderedDict([('value', 98.1), ('unit', '%'), ('symbol', 'F2q')])), ('initialization_fidelity', OrderedDict([('value', 99.7), ('unit', '%'), ('symbol', 

In [12]:
calibration

{'url': 'https://api.quantum-inspire.com/calibration/109006/',
 'backend': 'https://api.quantum-inspire.com/backends/8/',
 'parameters': {'system': {'fridge_temperature': {'value': '20.9',
    'unit': 'mK',
    'symbol': 'Fridge temperature'},
   'last_calibration_date': {'value': '04/11/2021 - 16:42 PM',
    'unit': 'iso-8601',
    'symbol': 'Calibration date'}},
  'qubits': {'q0': {'t1': {'value': 1.7341880614618646e-05,
     'unit': 's',
     'symbol': 'T1',
     'decimals': 1},
    't2star': {'value': 2.6163761239800992e-05,
     'unit': 's',
     'symbol': 'T2e',
     'decimals': 1},
    'single_qubit_gate_fidelity': {'value': 99.9,
     'unit': '%',
     'symbol': 'F1q'},
    'two_qubit_gate_fidelity': {'value': 98.1, 'unit': '%', 'symbol': 'F2q'},
    'initialization_fidelity': {'value': 99.7, 'unit': '%', 'symbol': 'Finit'},
    'readout_fidelity': {'value': 94.8, 'unit': '%', 'symbol': 'FR/O'}},
   'q1': {'t1': {'value': 1.7563309792567438e-05,
     'unit': 's',
     'symbol':

In [46]:
s = CalibrationViewer(calibration)
s.show_system_parameters()
s.show_qubit_parameters()
s.show_calibration_field('last_calibration_date')

try:
    c = s.show_calibration_field('readout_error_calibration')
except KeyError:
    print('readout error calibration does not exist')

s

System:

fridge_temperature: 20.9

last_calibration_date: 04/11/2021 - 16:42 PM

Qubit: q0

t1: 1.7341880614618646e-05

...

04/11/2021 - 16:42 PM

other fields:

{'value', 'unit', 'symbol'}

readout error calibration does not exist


<calibration of backend https://api.quantum-inspire.com/backends/8/ from time 04/11/2021 - 16:42 PM>

In [45]:
# TODO: pass standard deviations in calibration values (the decimals field is not suitable)

<calibration of backend https://api.quantum-inspire.com/backends/8/ from time 04/11/2021 - 16:42 PM>

In [44]:
from pprint import pprint
# pprint(str(s))
str(s)

'{"url": "https://api.quantum-inspire.com/calibration/109006/", "backend": "https://api.quantum-inspire.com/backends/8/", "parameters": {"system": {"fridge_temperature": {"value": "20.9", "unit": "mK", "symbol": "Fridge temperature"}, "last_calibration_date": {"value": "04/11/2021 - 16:42 PM", "unit": "iso-8601", "symbol": "Calibration date"}}, "qubits": {"q0": {"t1": {"value": 1.7341880614618646e-05, "unit": "s", "symbol": "T1", "decimals": 1}, "t2star": {"value": 2.6163761239800992e-05, "unit": "s", "symbol": "T2e", "decimals": 1}, "single_qubit_gate_fidelity": {"value": 99.9, "unit": "%", "symbol": "F1q"}, "two_qubit_gate_fidelity": {"value": 98.1, "unit": "%", "symbol": "F2q"}, "initialization_fidelity": {"value": 99.7, "unit": "%", "symbol": "Finit"}, "readout_fidelity": {"value": 94.8, "unit": "%", "symbol": "FR/O"}}, "q1": {"t1": {"value": 1.7563309792567438e-05, "unit": "s", "symbol": "T1", "decimals": 1}, "t2star": {"value": 3.6048567025453444e-05, "unit": "s", "symbol": "T2e"